In [ ]:
using Dates
using BenchmarkTools
using Fastback

In [ ]:
function run_backtest()
    # create instrument
    inst = Instrument(1, "AAPL")
    insts = [inst]

    # market data (order books)
    data = MarketData(insts)
    book = data.order_books[inst.index]

    # create trading account
    acc = Account(insts, 100_000.0)

    dt = DateTime(2018, 1, 2, 9, 30, 0)
    for i = 1:100_000
        ba = BidAsk(dt, 100.0 + i / 1000, 102 + i / 1000)
        update_book!(book, ba)
        update_account!(acc, data, inst)
        execute_order!(acc, book, Order(inst, 1.0, dt))
        execute_order!(acc, book, Order(inst, -1.0, dt))
    end
end

# w/ delayed orders min     = 14.4 ms
# direct placement          = 5.4 ms
@benchmark run_backtest() evals = 3 samples = 40

In [ ]:
using ProfileView
ProfileView.@profview map(i -> run_backtest(), 1:100)

In [ ]:
@benchmark map(i -> DateTime(0), 1:100_000)